In [1]:
import uproot

In [2]:
import uproot
import awkward
import numpy as np
from scipy.sparse import csr_matrix, find
from scipy.spatial import cKDTree
from tqdm import tqdm_notebook as tqdm


import os
import os.path as osp

print(os.environ['GNN_TRAINING_DATA_ROOT'])

#fname = osp.join(os.environ['GNN_TRAINING_DATA_ROOT'], 'single_taus/root/partGun_PDGid15_x1000_Pt3.0To100.0_NTUP_1.root')
fname = osp.join(os.environ['GNN_TRAINING_DATA_ROOT'], 'single_taus/root/partGun_PDGid15_x1000_Pt3.0To100.0_NTUP_9.root')

print(type(fname))

test = uproot.open(fname)['ana']['hgc']

%load_ext autoreload
%autoreload 2

/home/lagray/training_data
<class 'str'>


In [3]:
from scipy.sparse import coo_matrix # to encode the cluster mappings
from sklearn.neighbors import NearestNeighbors
from datasets.graph import Graph
from datasets.graph import graph_to_sparse, save_graph

sim_indices = awkward.fromiter(test['simcluster_hits_indices'].array())
sim_indices = sim_indices[sim_indices > -1].compact()
sim_energy = test['simcluster_energy'].array()
sim_pid = test['simcluster_pid'].array()

rechit_layer = test['rechit_layer'].array()
rechit_time = test['rechit_time'].array()
rechit_energy = test['rechit_energy'].array()

rechit_x = test['rechit_x'].array()
rechit_y = test['rechit_y'].array()
rechit_z = test['rechit_z'].array()

In [11]:
def get_category(pid):
    cats = np.zeros_like(pid) # 1 are hadrons
    cats[(pid == 22) | (np.abs(pid) == 11) | (pid == 111)] = 1 # 2 are EM showers
    cats[np.abs(pid) == 13] = 2 #3 are MIPs
    return (cats+1) # category zero are the noise hits

def get_features(ievt, mask):
    x = rechit_x[ievt][mask]
    y = rechit_y[ievt][mask]
    layer = rechit_layer[ievt][mask]
    time = rechit_time[ievt][mask]
    energy = rechit_energy[ievt][mask]    
    return np.stack((x,y,layer,time,energy)).T.astype(np.float32)

def get_neighbours(coords, map_idx, cluster_truth):
    nbrs = NearestNeighbors(algorithm='kd_tree').fit(coords)
    nbrs_sm = nbrs.kneighbors_graph(coords, 26)
    nbrs_sm.setdiag(0) #remove self-loop edges
    nbrs_sm.eliminate_zeros() 
    nbrs_sm = nbrs_sm + nbrs_sm.T
    pairs_sel = np.array(nbrs_sm.nonzero()).T
    data_sel = np.ones(pairs_sel.shape[0])
        
    #print(data_sel.shape)    
    #print(cluster_truth.shape)
    
    
    #map to absolute index
    #print('relative indices',pairs_sel)
    pairs_sel_abs = map_idx[pairs_sel]
    #print('absolute indices',pairs_sel_abs)
        
    #get the types of the clusters for these edges
    incoming = cluster_truth[pairs_sel_abs[:,1],:]    
    outgoing = cluster_truth[pairs_sel_abs[:,0],:]

    #print('truth shape',incoming.shape)
    #print('truth shape',outgoing.shape)    
    
    #determine determine all edges where each edge
    #has the same non-zero category
    hads = (incoming == 1).multiply(outgoing == 1)
    ems = (incoming == 2).multiply(outgoing == 2)
    mips = (incoming == 3).multiply(outgoing == 3)
    
    #print('hads',hads.todense())
    #print('ems',ems.todense())
    #print('mips',mips.todense())
    
    tot = (hads + ems + mips).nonzero()

    print('tot',np.unique(tot[1],return_counts=True))
    
    #prepare the input and output matrices (already need to store sparse)
    r_shape = (coords.shape[0],pairs_sel.shape[0])
    eye_edges = np.arange(pairs_sel.shape[0])
    
    R_i = csr_matrix((data_sel,(pairs_sel[:,1],eye_edges)), r_shape, dtype=np.uint8)
    R_o = csr_matrix((data_sel,(pairs_sel[:,0],eye_edges)), r_shape, dtype=np.uint8)
    
    # if you address the incoming edge by the outgoing index then the edge connects two
    # hits in the same sim-cluster
    y = np.zeros(shape=pairs_sel.shape[0], dtype=np.int8)
    truth = np.squeeze(np.asarray(incoming[tot[0],tot[1]]))
    if tot[0].size > 0 and tot[1].size > 0:
        y[tot[0]] = truth
    
    return R_i, R_o, y

for i in tqdm(range(rechit_z.size),desc='events processed'): #
        
    if i > 20: break    
    
    cluster_cats = get_category(sim_pid[i])
    
    print(i,sim_pid[i])
    #print(cluster_cats)
        
    sim_indices_cpt = awkward.fromiter(sim_indices[i])
    hits_in_clus = sim_indices_cpt.flatten()
    hit_to_clus = sim_indices_cpt.parents
    #print(hit_to_clus)
    #print(np.unique(hit_to_clus,return_counts=True))
    # 0 = invalid edge, 1 = hadronic edge, 2 = EM edge, 3 = MIP edge 
    cats_per_hit = cluster_cats[hit_to_clus]
    
    #print(cats_per_hit)
    
    #print(hits_in_clus.shape, hit_to_clus.shape, cats_per_hit.shape)
    
    hit_truth = np.stack((hits_in_clus, hit_to_clus, cats_per_hit)).T
    #hit_truth = hit_truth[np.argsort(hit_truth[:,0])]
    
    #print('raw hit truth',hit_truth)
    
    hits_to_clusters = csr_matrix((hit_truth[:,2], (hit_truth[:,0], hit_truth[:,1])),
                                  (rechit_z[i].size, np.max(hit_to_clus)+1))    
    
    #print('sparse hit truth',hits_to_clusters.todense())

    pos_mask = (rechit_z[i] > 0)
    neg_mask = ~pos_mask
    
    rechit_indices = np.arange(rechit_z[i].size)
    
    pos_feats = get_features(i, pos_mask)
    neg_feats = get_features(i, neg_mask)
    
    #print(rechit_indices.shape, pos_mask.shape, neg_mask.shape)
    
    #print(rechit_indices, rechit_indices.shape)    
    
    pos_indices = rechit_indices[pos_mask]
    neg_indices = rechit_indices[neg_mask]
    
    #print(pos_indices, pos_indices.shape)
    #print(neg_indices, neg_indices.shape)
    
    pos_coords = pos_feats[:,0:3]
    neg_coords = neg_feats[:,0:3]
            
    # 0 = invalid edge, 1 = hadronic edge, 2 = EM edge, 3 = MIP edge    
    pos_Ri, pos_Ro, pos_y = get_neighbours(pos_coords, pos_indices, hits_to_clusters)
    neg_Ri, neg_Ro, neg_y = get_neighbours(neg_coords, neg_indices, hits_to_clusters)
    
    
    
    pos_graph = Graph(pos_feats, pos_Ri, pos_Ro, pos_y, simmatched = np.array([]))
    print(np.unique(pos_y,return_counts=True))
    neg_graph = Graph(neg_feats, neg_Ri, neg_Ro, neg_y, simmatched = np.array([]))
    print(np.unique(neg_y,return_counts=True))
    
    outbase = fname.split('/')[-1].replace('.root','')
    outdir = "/".join(fname.split('/')[:-2]) + "/test" + "/raw"
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    # for UnnormalizedEdgeNet
    save_graph(pos_graph, '%s/%s_hgcal_graph_pos_evt%d.npz'%(outdir,outbase,i))
    save_graph(neg_graph, '%s/%s_hgcal_graph_neg_evt%d.npz'%(outdir,outbase,i))
        

0 [ 13 211  11 -11  11 -11]
tot (array([0], dtype=int32), array([638]))
tot (array([1, 2, 3, 4, 5], dtype=int32), array([10294,  3272,    30,  5854,   202]))
(array([0, 3], dtype=int8), array([552788,    638]))
(array([0, 1, 2], dtype=int8), array([573536,  10294,   9066]))
1 [-211  211 -211   11   11  -11]
tot (array([0, 1, 2], dtype=int32), array([1606, 1672, 5544]))
tot (array([3, 4, 5], dtype=int32), array([3922,   20,   12]))
(array([0, 1], dtype=int8), array([557154,   8822]))
(array([0, 2], dtype=int8), array([555006,   3954]))
2 [  11   22   11  -11   22 -211   22   22   22  211   22   22 2212  211
  211 -211   11   11  -11  -11   11  -11]
tot (array([0, 1, 2, 3, 4], dtype=int32), array([18414,   204,  2452,  3992,   460]))
tot (array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
      dtype=int32), array([  404,   880,    20,    90,    30,   284,   254,     6,    62,
        3206,  5842,  5454,     4,     2,  2430, 12426,  3856]))
(array([0, 2], dtype=i

18 [2112  211 2212 -211 -211  -11  -11   22   22]
tot (array([1, 2, 3, 4], dtype=int32), array([  54,    2, 1344,  978]))
tot (array([7, 8], dtype=int32), array([  72, 2382]))
(array([0, 1], dtype=int8), array([554056,   2378]))
(array([0, 2], dtype=int8), array([550612,   2454]))
19 [-211  -11  -11  -11   22   11  -11   11  -11   22   11  -11   11  -11]
tot (array([ 0, 10, 11, 12, 13], dtype=int32), array([29792,  2372,  1856,   120,  4674]))
tot (array([1, 3, 4, 5, 6, 8, 9], dtype=int32), array([  12, 2700, 4642,  568,  676,   18,   72]))
(array([0, 1, 2], dtype=int8), array([566072,  29630,   8126]))
(array([0, 2], dtype=int8), array([563682,   8578]))
20 [ -11   11   22   11  -11   22   11   11  -11 -211  211   11  -11   11]
tot (array([0, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32), array([10786,    22,  1544,   390,   192,    58,  1002,    62,  4778]))
tot (array([10, 11, 12, 13], dtype=int32), array([2874,  560, 1126, 4670]))
(array([0, 1, 2], dtype=int8), array([564016,   4778,  1398

In [9]:
%matplotlib inline

import matplotlib.pyplot as plt

def draw_sample(X, Ri, Ro, y, out,
                cmap='bwr_r', 
                skip_false_edges=True,
                alpha_labels=False, 
                sim_list=None): 
    
    #let's draw only the non-noise edges
    out_mask = out > 0
    Ri = Ri[out_mask]
    Ro = Ro[out_mask]
    good_outs = out[out_mask]
    
    # Select the i/o node features for each segment    
    feats_o = X[Ro]
    feats_i = X[Ri]    
    # Prepare the figure
    fig, (ax0,ax1) = plt.subplots(1, 2, figsize=(20,12))
    cmap = plt.get_cmap(cmap)
    
    
    #if sim_list is None:    
        # Draw the hits (layer, x, y)
    #    ax0.scatter(X[:,0], X[:,2], c='k')
    #    ax1.scatter(X[:,1], X[:,2], c='k')
    #else:        
    ax0.scatter(X[:,0], X[:,2], c='k')
    ax1.scatter(X[:,1], X[:,2], c='k')
        
    # Draw the segments    
    if out is not None:
        #t = tqdm.tqdm()
        color_map = {1: dict(c='blue'),
                     2: dict(c='red'),
                     3: dict(c='orange')}
        for j in range(good_outs.shape[0]):
            seg_args = color_map[out[j]]            

            ax0.plot([feats_o[j,0], feats_i[j,0]],
                     [feats_o[j,2], feats_i[j,2]], '-', **seg_args)
            ax1.plot([feats_o[j,1], feats_i[j,1]],
                     [feats_o[j,2], feats_i[j,2]], '-', **seg_args)
    else:
        t = tqdm.tqdm(range(y.shape[0]))
        for j in t:
            if y[j]:
                seg_args = dict(c='b', alpha=0.4)
            elif not skip_false_edges:
                seg_args = dict(c='black', alpha=0.4)
            else: continue
                
            ax0.plot([feats_o[j,0], feats_i[j,0]],
                     [feats_o[j,2], feats_i[j,2]], '-', **seg_args)
            ax1.plot([feats_o[j,1], feats_i[j,1]],
                     [feats_o[j,2], feats_i[j,2]], '-', **seg_args)
        
    # Adjust axes
    ax0.set_xlabel('$x$ [cm]')
    ax1.set_xlabel('$y$ [cm]')
    ax0.set_ylabel('$layer$ [arb]')
    ax1.set_ylabel('$layer$ [arb]')
    plt.tight_layout()
    return fig;

In [ ]:
X = pos_graph.x.numpy()
index = pos_graph.edge_index.numpy().T
Ro = index[:,0]
Ri = index[:,1]
y = data.y.numpy()

out = np.argmax(pred_edges_np,axis=-1)

draw_sample(X, Ri, Ro, y, y)